<a href="https://colab.research.google.com/github/PharahMain/Flight-Delay-Predictor/blob/master/flightstats_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [3]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 38.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=7c3a54aa60dcacd8d9d16ee27f8760565011d3f13b2b02169da459e5cce1750f
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [5]:
# Point Colaboratory to Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col, create_map, lit
from pyspark.sql.functions import month, dayofweek, weekofyear, hour, minute
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType, TimestampType

from datetime import datetime, date, timedelta
from itertools import chain
import random
import time

In [0]:
#ARR_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/{date.today()}.json"
# for testing
ARR_JSON_PATH = "/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/2020-01-17.json"

#DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/{date.today()}.json"
DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/2020-01-17.json"

#WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/{date.today()}.json"
WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/2020-01-17.json"

APP_NAME = 'Flight Delay Predictor'
SPARK_URL = 'local[*]'
RANDOM_SEED = 263646
TRAINING_DATA_RATIO = 0.7

In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [9]:
df_arr = spark.read.json(ARR_JSON_PATH)
df_arr.show(5)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|    airportResources|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          codeshares|              delays|departureAirportFsCode|       departureDate|divertedAirportFsCode|     flightDurations|   flightEquipment|  flightId|flightNumber|    operationalTimes|            schedule|status|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|      [, TBIT,, C9,]|                 LAX|[2020-01-17T13:10...|           QR|     [[3515, UL, L]]| 

The data in a little more detail.

In [0]:
#df_arr.head(5)

Let's see how many flights arrived at LAX in the past 24 hours.

In [0]:
df_arr.count()

337

Let's first drop all the unnecessary columns.

In [0]:
df_arr = df_arr.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')
#df_arr.show(5)

In [0]:
df_arr.printSchema()

root
 |-- arrivalAirportFsCode: string (nullable = true)
 |-- arrivalDate: struct (nullable = true)
 |    |-- dateLocal: string (nullable = true)
 |    |-- dateUtc: string (nullable = true)
 |-- carrierFsCode: string (nullable = true)
 |-- delays: struct (nullable = true)
 |    |-- arrivalGateDelayMinutes: long (nullable = true)
 |    |-- arrivalRunwayDelayMinutes: long (nullable = true)
 |    |-- departureGateDelayMinutes: long (nullable = true)
 |    |-- departureRunwayDelayMinutes: long (nullable = true)
 |-- departureAirportFsCode: string (nullable = true)
 |-- flightDurations: struct (nullable = true)
 |    |-- blockMinutes: long (nullable = true)
 |    |-- scheduledAirMinutes: long (nullable = true)
 |    |-- scheduledBlockMinutes: long (nullable = true)
 |    |-- scheduledTaxiInMinutes: long (nullable = true)
 |    |-- scheduledTaxiOutMinutes: long (nullable = true)
 |    |-- taxiOutMinutes: long (nullable = true)
 |-- flightEquipment: struct (nullable = true)
 |    |-- actualEq

Let's explore the dataframe a little bit here.

In [0]:
df_arr.groupBy('carrierFsCode').count().orderBy('count', ascending=False).show()

+-------------+-----+
|carrierFsCode|count|
+-------------+-----+
|           DL|   48|
|           AA|   45|
|           OO|   44|
|           WN|   42|
|           CP|   30|
|           UA|   25|
|           AS|   24|
|           NK|    6|
|           B6|    6|
|           QX|    5|
|           AC|    4|
|           Y4|    3|
|           WS|    3|
|           BA|    3|
|           QF|    3|
|           K4|    2|
|           BR|    2|
|          BAI|    2|
|           NZ|    2|
|           1I|    2|
+-------------+-----+
only showing top 20 rows



Let's see what the busiest time of the day is for LAX next. But before we do that, I'm going to separate out the arrival time from 'arrivalDate'.

In [0]:
# first need to extract the date info from the arrivalDate struct
df_arr = df_arr.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
#df_arr.select('localDate').show(5)

df_arr = df_arr.withColumn('localTimeStamp', df_arr['localDate'].cast(TimestampType()))
#df_arr.select('localTimeStamp').show(5)

Give each time increment its own column.

In [0]:
#df_arr.withColumn('date', col('localTimeStamp').cast('date')).withColumn('hour', hour(col('localTimeStamp'))).withColumn('minute', minute(col('localTimeStamp'))).show(5)

Since travel tends to change with the time of year, let's also add on weekOfYear and dayOfWeek as well.

In [0]:
#df_arr.withColumn('weekOfYear', weekofyear(col('localTimeStamp'))).withColumn('dayOfWeek', dayofweek(col('localTimeStamp'))).show(5)

Delays column, the column that contains our target data, is an array of four values. They are minutes of delay in different stages of the flight but, since travelers don't really care where the delays occur, but rather how long the delays are, let's sum up the numbers to get one total delayed minutes.

In [0]:
# a df with just the delay data
delay_df = df_arr.select('delays.*').na.fill(0)
#delay_df.show(5)

In [0]:
# create a new column in main arrival df with total delay minutes
delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

#delay_df.show(5)


Now let's merge this table with the main table.

In [0]:
# In order to join two dataframes, we need to create an index column to join the two 
# dataframes on
df_arr = df_arr.select('*').withColumn('id', monotonically_increasing_id())

df_arr = df_arr.join(delay_df, col('id') == col('d_id'), how='leftouter').drop('d_id')
#df_arr.show(5)

Now let's do the same for flightDurations and flightEquipment.

In [0]:
eq_df = df_arr.select('flightEquipment.*').drop('tailNumber')
#eq_df.show(5)

When it comes to equipment, when it is changed it might cause a delay so I'm going to drop the tailNumber and create another column that shows whether there was a change in equipment.

In [0]:
eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0)
    .otherwise(1))

eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())
#eq_df.show(5)

And lastly, let's just some up the flight durations.

In [0]:
fd_df = df_arr.select('flightDurations.*').na.fill(0)
fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())
#fd_df.show(5)

Put all the tables together.

In [0]:
df_arr = df_arr.join(eq_df, col('id') == col('e_id'), how='leftouter').drop('e_id').join(fd_df, col('id') == col('f_id'), how='leftouter').drop('f_id')
#df_arr.orderBy('id').show(5)

In [0]:
df_arr.count()

337

We now need to repeat the same process for the departing flights data. It might be better if I wrote a function that does this given a dataframe.

In [0]:
def format_flight_data(df):
    # lets' first index the df since we know we will be needing it later
    df = df.select('*').withColumn('id', monotonically_increasing_id())

    ##### timestamp #####
    # first need to extract the date info from the arrivalDate struct
    df = df.withColumn('localDate', col('arrivalDate').getField('dateLocal'))
    df = df.withColumn('localTimeStamp', df['localDate'].cast(TimestampType()))
    df = df.withColumn('date', col('localTimeStamp').cast('date'))
    df = df.withColumn('hour', hour(col('localTimeStamp')))
    df = df.withColumn('minute', minute(col('localTimeStamp')))
    df = df.withColumn('weekOfYear', weekofyear(col('localTimeStamp')))
    df = df.withColumn('dayOfWeek', dayofweek(col('localTimeStamp')))

    ##### delays #####
    # fill all null values with 0
    delay_df = df.select('delays.*').na.fill(0)
    # create a new column in main arrival df with total delay minutes
    delay_df = delay_df.withColumn('totalDelay', sum(delay_df[col] for col in delay_df.columns))
    delay_df = delay_df.select('*').withColumn('d_id', monotonically_increasing_id())

    ##### equipment #####
    eq_df = df.select('flightEquipment.*').drop('tailNumber')
    eq_df = eq_df.withColumn('eqChanged', F.when(F.col('actualEquipmentIataCode') == F.col('scheduledEquipmentIatacode'), 0).otherwise(1))
    eq_df = eq_df.select('*').withColumn('e_id', monotonically_increasing_id())

    ##### duration #####
    fd_df = df.select('flightDurations.*').na.fill(0)
    fd_df = fd_df.withColumn('totalDuration', sum(fd_df[col] for col in fd_df.columns))
    fd_df = fd_df.select('*').withColumn('f_id', monotonically_increasing_id())

    # join all dataframes
    formatted_df = df.join(delay_df, col('id')==col('d_id'), how='leftouter').drop('d_id')
    formatted_df = formatted_df.join(eq_df, col('id')==col('e_id'), how='leftouter').drop('e_id')
    formatted_df = formatted_df.join(fd_df, col('id')==col('f_id'), how='leftouter').drop('f_id')

    # drop extraneous columns
    formatted_df = formatted_df.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status',
            'flightId', 'flightNumber')

    return formatted_df

    

In [11]:
df_arr_fmtd = format_flight_data(df_arr)
df_arr_fmtd.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|s

Nice. Now let's format the departing dataframe.

In [12]:
df_dep = spark.read.json(DEP_JSON_PATH)
df_dep_fmtd = format_flight_data(df_dep)
df_dep_fmtd.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDura

Now that we have a formatted dataframe for both arriving and departing flights, we need to join the weather conditions for each flight. The data from the DarkSky API contains within it a timestamp but it is in UNIX time. So now we just need to convert it so that it matches up with the formatted dataframe and do an inner join.

In [13]:
df_weath = spark.read.json(WEATH_JSON_PATH)
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|
|               53.2|      0.88|   49.85|    0.88|               rain|303.5|         0.0262|            

In [14]:
df_weath = df_weath.withColumn('f_time', df_weath['time'].cast(TimestampType()))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|2020-01-17 08:00:00|
|               53.2|   

Hmmm. Since the weather data is taken hourly, we now just need to match upto the hour.

In [15]:
# separate date and hour
df_weath = df_weath.withColumn('w_date', col('f_time').cast('date'))
df_weath = df_weath.withColumn('w_hour', hour(col('f_time')))
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|             f_time|    w_date|w_hour|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+-------------------+----------+------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85| 

Now all that's left to do is inner join the weather data on both the date and hour columns.

In [16]:
df_arr_wthr = df_arr_fmtd.join(df_weath, (col('date')==col('w_date'))&(col('hour')==col('w_hour')), how='inner')
df_arr_wthr = df_arr_wthr.drop('time', 'w_date', 'w_hour')
df_arr_wthr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrival

In [17]:
df_dep_wthr = df_dep_fmtd.join(df_weath, (col('date')==col('w_date')) & (col('hour')==col('w_hour')), how='inner')
df_dep_wthr = df_dep_wthr.drop('time', 'w_date', 'w_hour')
df_dep_wthr.show(5)

+--------------------+--------------------+-------------+----------------+----------------------+-----------------+------------------+---+--------------------+-------------------+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+-------------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          delays|departureAirportFsCode|  flightDurations|   flightEquipment| id|           localDate|     localTimeStamp|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes

Next, I think I will select only the feature and target columns.

In [18]:
df_arr_final = df_arr_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment', 
                               'localDate', 'localTimeStamp', 'icon', 'f_time')
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|

In [19]:
# rinse and repeat for the departing flights
df_dep_final = df_dep_wthr.drop('arrivalDate', 'delays', 'flightDurations', 'flightEquipment',
                                'localDate', 'localTimeStamp', 'icon', 'f_time')
df_dep_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+-------------------+---------------------+----------------------+-----------------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|hu

While we have the columns narrowed down, we still have a long way to go to be ready to build a model. Since we will be using linear regression as one of the methods, we have to encode all the categorical variables.

In [20]:
df_arr_final.printSchema()

root
 |-- arrivalAirportFsCode: string (nullable = true)
 |-- carrierFsCode: string (nullable = true)
 |-- departureAirportFsCode: string (nullable = true)
 |-- id: long (nullable = false)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- weekOfYear: integer (nullable = true)
 |-- dayOfWeek: integer (nullable = true)
 |-- arrivalGateDelayMinutes: long (nullable = true)
 |-- arrivalRunwayDelayMinutes: long (nullable = true)
 |-- departureGateDelayMinutes: long (nullable = true)
 |-- departureRunwayDelayMinutes: long (nullable = true)
 |-- totalDelay: long (nullable = true)
 |-- actualEquipmentIataCode: string (nullable = true)
 |-- scheduledEquipmentIataCode: string (nullable = true)
 |-- eqChanged: integer (nullable = true)
 |-- blockMinutes: long (nullable = true)
 |-- scheduledAirMinutes: long (nullable = true)
 |-- scheduledBlockMinutes: long (nullable = true)
 |-- scheduledTaxiInMinutes: long (nullable = true)
 |-- sc

In [54]:
# let's first create a dictionary of all the carriers present in the dataframe
# get unique values from column
unique_airlines = df_arr_final.select('carrierFsCode').distinct()

# make a list of all airlines
airline_list = [row.carrierFsCode for row in unique_airlines.collect()]

# create a random sampling of integer ids
random_codes = random.sample(range(1000), len(airline_list))

# assign each random int id to an airline
airline_dict = dict(zip(airline_list, random_codes))
print(airline_dict)


{'XOJ': 136, 'CI': 774, 'BAI': 80, 'UA': 264, 'NK': 715, 'AA': 955, 'AV': 508, 'QF': 828, 'CZ': 843, 'B6': 194, 'CX': 202, 'NH': 915, 'CV': 495, 'SU': 991, 'LX': 143, 'DL': 773, 'CA': 683, 'MW': 288, 'BR': 301, 'OO': 792, 'F9': 846, 'VB': 504, 'LH': 815, 'CM': 104, 'Y4': 62, 'VS': 703, 'HA': 724, 'BA': 418, 'LP': 388, 'EI': 908, 'WS': 605, 'DI*': 148, 'LO': 507, 'MU': 125, '5X': 921, 'G4': 290, 'MF': 671, 'NZ': 409, 'CP': 585, 'QR': 885, 'OZ': 637, 'KE': 325, '1I': 306, 'AS': 898, 'AC': 102, 'QX': 248, 'K4': 557, 'TK': 184, 'PR': 393, 'TA': 417, 'SQ': 841, 'AF': 186, 'WN': 157, 'L1E': 525}


In [25]:
total_flts = df_arr_final.count()
unique_from_stn = df_arr_final.select('departureAirportFsCode').distinct()
from_stn_total = unique_from_stn.count()
total_alines = len(airline_list)
arr_airpt = df_arr_final.first().arrivalAirportFsCode

print(f'There were {total_flts} flights from {from_stn_total} different stations operated by {total_alines} different airlines arriving at {arr_airpt} in the past 24 hours.')

There were 337 flights from 123 different stations operated by 54 different airlines arriving at LAX in the past 24 hours.


Now that we have a dictionary, let's apply it to our dataframe.

In [26]:
map_exp = create_map([lit(x) for x in chain(*airline_dict.items())])

df_arr_final = df_arr_final.withColumn('a_code', map_exp.getItem(col('carrierFsCode')))
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+----------+--------+-------------+-----------+-------+----------+-----------+--------+---------+------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutM

Looking through the dataframe, we have a couple more columns that needs to go under the same process to encode.

In [27]:
# originating stations
from_stn_list = [row.departureAirportFsCode for row in unique_from_stn.collect()]
random_codes_2 = random.sample(range(1001, 2000), len(from_stn_list))
from_stn_dict = dict(zip(from_stn_list, random_codes_2))
print(from_stn_dict)

{'MSY': 1345, 'GEG': 1991, 'EUG': 1512, 'OAK': 1518, 'BNE': 1765, 'MSN': 1220, 'DCA': 1887, 'QRO': 1574, 'RDM': 1833, 'PEK': 1464, 'CMH': 1992, 'CDG': 1046, 'SIN': 1371, 'IAH': 1634, 'MZT': 1085, 'SYD': 1769, 'HNL': 1380, 'SJC': 1197, 'BUF': 1550, 'PVG': 1917, 'AUS': 1728, 'MUC': 1614, 'MEX': 1139, 'ZIH': 1154, 'RNO': 1429, 'JAC': 1217, 'IPL': 1881, 'TUL': 1076, 'BOS': 1191, 'EWR': 1238, 'DOH': 1121, 'LAS': 1306, 'YEG': 1214, 'LIR': 1567, 'BCN': 1997, 'XNA': 1933, 'YYZ': 1003, 'DEN': 1734, 'BJX': 1541, 'IAD': 1638, 'BOI': 1772, 'SBA': 1385, 'SJD': 1494, 'SEA': 1413, 'MNL': 1715, 'ICN': 1704, 'MCI': 1107, 'WAW': 1178, 'CLT': 1717, 'BNA': 1093, 'PAE': 1447, 'YVR': 1163, 'CUN': 1325, 'ABQ': 1328, 'SVO': 1377, 'SDF': 1183, 'MEL': 1213, 'SCK': 1802, 'DAL': 1896, 'MRY': 1216, 'PDX': 1757, 'PVR': 1938, 'LGW': 1805, 'MIA': 1743, 'MFR': 1852, 'LHR': 1663, 'DGO': 1526, 'PRC': 1442, 'SAL': 1976, 'OKC': 1607, 'YYC': 1905, 'SMF': 1335, 'PTY': 1037, 'PHX': 1432, 'STL': 1475, 'DFW': 1364, 'MMH': 1028

In [0]:
#df_arr_final.columns

In [28]:
# aircrafts(ac)
ac_actual = df_arr_final.select('actualEquipmentIataCode').na.fill('None').distinct()
ac_sched = df_arr_final.select('scheduledEquipmentIataCode').na.fill('None').distinct()

ac_actual_list = [row.actualEquipmentIataCode for row in ac_actual.collect()]
ac_sched_list = [row.scheduledEquipmentIataCode for row in ac_sched.collect()]

ac_combo_list = list(set(ac_actual_list + ac_sched_list))

random_codes_3 = random.sample(range(2001, 3000), len(ac_combo_list))

ac_dict = dict(zip(ac_combo_list, random_codes_3))
print(ac_dict)

print(f'A total of {len(ac_combo_list)} different types of aircraft operating today.')

{'73G': 2670, '752': 2362, '7S8': 2732, '753': 2404, '388': 2093, 'PL2': 2904, '74F': 2631, '75T': 2055, 'CR7': 2786, '77F': 2225, 'CN2': 2293, '321': 2637, '76W': 2010, '773': 2158, '74Y': 2925, 'CNA': 2380, 'None': 2418, '73H': 2250, '32N': 2773, '73J': 2663, '788': 2309, 'CJL': 2434, '75W': 2873, 'E7W': 2476, '772': 2284, '77L': 2820, '73R': 2642, '32B': 2824, '359': 2916, '319': 2875, '739': 2041, '32Q': 2562, '763': 2295, '741': 2766, '738': 2466, '32S': 2990, '32A': 2672, 'CR2': 2247, '757': 2729, '31B': 2449, '75D': 2806, '76V': 2065, '320': 2267, '346': 2714, '332': 2793, '330': 2367, 'E75': 2752, '77W': 2550, '744': 2677, '73W': 2830, '747': 2259, '77X': 2861, '781': 2152, '789': 2656, '74N': 2275, 'CNJ': 2245, 'CRJ': 2366}
A total of 57 different types of aircraft operating today.


In [29]:
# precipitation
df_arr_final.select('precipType').distinct().show()


+----------+
|precipType|
+----------+
|      null|
|      rain|
+----------+



It looks like we can just drop this column altogether.

In [0]:
df_arr_final = df_arr_final.drop('precipType', 'summary')

In [37]:
df_arr_final.show(1)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOut

In [0]:
map_exp_2 = create_map([lit(x) for x in chain(*from_stn_dict.items())])
map_exp_3 = create_map([lit(x) for x in chain(*ac_dict.items())])

In [0]:
df_arr_final = df_arr_final.withColumn('fm_code', map_exp_2.getItem(col('departureAirportFsCode')))
df_arr_final = df_arr_final.withColumn('aac_code', map_exp_3.getItem(col('actualEquipmentIataCode')))
df_arr_final = df_arr_final.withColumn('sac_code', map_exp_3.getItem(col('scheduledEquipmentIataCode')))

In [34]:
df_arr_final.show(5)

+--------------------+-------------+----------------------+---+----------+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+-----------------------+--------------------------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|arrivalAirportFsCode|carrierFsCode|departureAirportFsCode| id|      date|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|actualEquipmentIataCode|scheduledEquipmentIataCode|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOut

Once the columns containing non-numerical values are dropped, we should finally have a dataframe we can model and predict with.

In [35]:
df_arr_num = df_arr_final.drop('arrivalAirportFsCode', 'carrierFsCode', 'departureAirportFsCode', 'date', 'id',
                               'actualEquipmentIataCode', 'scheduledEquipmentIataCode')
df_arr_num.show(5)

+----+------+----------+---------+-----------------------+-------------------------+-------------------------+---------------------------+----------+---------+------------+-------------------+---------------------+----------------------+-----------------------+--------------+-------------+-------------------+----------+--------+--------+-----+---------------+-----------------+--------+-----------+-------+----------+-----------+--------+---------+------+-------+--------+--------+
|hour|minute|weekOfYear|dayOfWeek|arrivalGateDelayMinutes|arrivalRunwayDelayMinutes|departureGateDelayMinutes|departureRunwayDelayMinutes|totalDelay|eqChanged|blockMinutes|scheduledAirMinutes|scheduledBlockMinutes|scheduledTaxiInMinutes|scheduledTaxiOutMinutes|taxiOutMinutes|totalDuration|apparentTemperature|cloudCover|dewPoint|humidity|ozone|precipIntensity|precipProbability|pressure|temperature|uvIndex|visibility|windBearing|windGust|windSpeed|a_code|fm_code|aac_code|sac_code|
+----+------+----------+--------

Just like we have been doing so far, let's process the the departing flights dataframe in summary fashion.

In [0]:
# function to encode each categorical variable
def get_dict(filtered_df):

    # complete dictionary to be returned
    comp_dict = {}

    # get categorical columns
    cat_cols = [c for c, t in filtered_df.dtypes if t == 'string']
    uniqs = {'airports': [], 'equipments': [], 'carriers': []}

    # separate each category into its own list
    for c in cat_cols:
        uniq = filtered_df.select(c).na.fill('None').distinct()
        u_list = [row[c] for row in uniq.collect()]

        if 'Airport' in c:
            uniqs['airports'] += u_list
        elif 'Equipment' in c:
            uniqs['equipments'] += u_list
        elif 'carrier' in c:
            uniqs['carriers'] += u_list

    # for each category get set of unique values and create dictionary
    for category in uniqs.keys():
        i = 0 
        u_vals = uniqs[category]
        u_set = list(set(u_vals))
        start, stop = i*1000, (i+1)*1000
        rnd_codes = random.sample(range(start, stop), len(u_set))
        comp_dict.update({category: dict(zip(u_set, rnd_codes))})
        i += 1

    print(f"There are a total of {filtered_df.count()} flights arriving from {len(comp_dict['airports'])} cities operated by {len(comp_dict['carriers'])} airlines using {len(comp_dict['equipments'])} different types of aircraft at LAX in the past 24 hours.")
    
    return comp_dict

In [76]:
print(get_dict(df_arr_final))

There are a total of 337 flights arriving from 123 cities operated by 54 airlines using 57 different types of aircraft at LAX in the past 24 hours.
{'airports': {'EWR': 521, 'BNA': 576, 'MUC': 69, 'QRO': 447, 'MSN': 96, 'TAO': 212, 'HNL': 761, 'PPT': 733, 'YYZ': 756, 'MFR': 42, 'MEM': 281, 'STS': 463, 'OMA': 443, 'DEN': 559, 'MCO': 735, 'GEG': 601, 'LIR': 305, 'SEA': 687, 'PAE': 152, 'MIA': 837, 'PHX': 271, 'JFK': 22, 'PVR': 420, 'RNO': 165, 'IAD': 31, 'SVO': 910, 'DCA': 241, 'PHL': 394, 'OKC': 792, 'ATL': 175, 'BOI': 554, 'TUS': 127, 'EGE': 714, 'SUN': 782, 'IAH': 972, 'MDW': 372, 'EUG': 693, 'BCN': 920, 'ASE': 673, 'SYD': 624, 'MEL': 947, 'LAX': 457, 'ELP': 786, 'ZRH': 89, 'DOH': 168, 'TRM': 292, 'DAL': 491, 'MSP': 137, 'MMH': 424, 'LGW': 285, 'SJD': 708, 'LIM': 258, 'BNE': 139, 'MEX': 118, 'TEB': 881, 'YEG': 939, 'AUS': 170, 'WAW': 999, 'YVR': 895, 'CAN': 470, 'HDN': 307, 'BUF': 400, 'TPE': 586, 'ABQ': 479, 'SAN': 968, 'IPL': 865, 'SMF': 878, 'BZN': 235, 'SAT': 286, 'SBA': 403, 'JAC

In [57]:
# function to purify joined data
def clean_joined(joined_df):
    # first gather all non-numerical data
    cat_cols = [c for c, t in joined_df.dtypes if t == 'string']

    # iterate through each column and create code dictionary
    for i, column in enumerate(cat_cols):
        this_dict = get_dict(joined_df, cat_cols[i], i)
        this_map_exp = create_map([lit(x) for x in chain(*this_dict.items())])
        joined_df = joined_df.withColumn(f'{column[:5]}_code', this_map_exp.getItem(col(column)))

    for i, column in enumerate(cat_cols):
        if 'Airport' in column:
            

    return joined_df

df_arr_num = clean_joined(df_arr_final)
df_arr_num.show(5)

{'HNL': 79, 'LAX': 517}
{'XOJ': 1228, 'CI': 1312, 'BAI': 1639, 'UA': 1200, 'NK': 1283, 'AA': 1271, 'AV': 1990, 'QF': 1619, 'CZ': 1468, 'B6': 1473, 'CX': 1344, 'NH': 1290, 'CV': 1946, 'SU': 1928, 'LX': 1775, 'DL': 1624, 'CA': 1837, 'MW': 1972, 'BR': 1174, 'OO': 1331, 'F9': 1807, 'VB': 1401, 'LH': 1873, 'CM': 1206, 'Y4': 1983, 'VS': 1488, 'HA': 1223, 'BA': 1470, 'LP': 1514, 'EI': 1390, 'WS': 1012, 'DI*': 1948, 'LO': 1090, 'MU': 1569, '5X': 1179, 'G4': 1145, 'MF': 1939, 'NZ': 1210, 'CP': 1839, 'QR': 1524, 'OZ': 1546, 'KE': 1824, '1I': 1618, 'AS': 1088, 'AC': 1816, 'QX': 1337, 'K4': 1099, 'TK': 1548, 'PR': 1037, 'TA': 1134, 'SQ': 1162, 'AF': 1633, 'WN': 1493, 'L1E': 1268}
{'MSY': 2705, 'GEG': 2594, 'EUG': 2009, 'OAK': 2238, 'BNE': 2001, 'MSN': 2796, 'DCA': 2240, 'QRO': 2348, 'RDM': 2525, 'PEK': 2512, 'CMH': 2568, 'CDG': 2435, 'SIN': 2436, 'IAH': 2573, 'MZT': 2562, 'SYD': 2329, 'HNL': 2975, 'SJC': 2701, 'BUF': 2738, 'PVG': 2269, 'AUS': 2416, 'MUC': 2970, 'MEX': 2394, 'ZIH': 2365, 'RNO': 236